In [4]:
import pandas as pd
import numpy as np

import scanpy as sc
import biolord

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.stats import ttest_rel
import anndata

In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_biolord_split_30.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_biolord_split_30.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_biolord_split_30.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [5]:
adata = anndata.concat((adata_train, adata_test, adata_ood), label="split", keys=["train", "test", "ood"])

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
adata.obsm

AxisArrays with keys: X_pca, cell_line_emb, ecfp, ecfp_cell_line, ecfp_cell_line_dose, ecfp_cell_line_dose_more_dose, ecfp_cell_line_logdose, ecfp_cell_line_logdose_more_dose

In [113]:
dose = adata.obs["dose"].astype("float") / np.max(adata.obs["dose"].astype("float")) # following 

In [117]:
adata.obsm["ecfp_dose"] = np.concatenate((adata.obsm["ecfp"], dose.values[:,None]), axis=1)

In [11]:
biolord.Biolord.setup_anndata(
    adata,
    ordered_attributes_keys=["ecfp_logdose"],
    categorical_attributes_keys=["cell_type"],
    retrieval_attribute_key=None,
)

INFO     Generating sequential column names                                                                        


In [27]:
module_params= dict(
decoder_width = 4096,
decoder_depth = 4,

attribute_dropout_rate = 0.1,
attribute_nn_width = 2048,
attribute_nn_depth = 2,

unknown_attribute_noise_param = 2e+1,
unknown_attribute_penalty = 1e-1,
gene_likelihood = "normal",
n_latent_attribute_ordered = 256,
n_latent_attribute_categorical = 3,
reconstruction_penalty = 1e+4,
use_batch_norm = False,
use_layer_norm = False,)

trainer_params=dict(
latent_lr = 1e-4,
latent_wd = 1e-4,
decoder_lr = 1e-4,
decoder_wd = 1e-4,
attribute_nn_lr = 1e-2,
attribute_nn_wd = 4e-8,
cosine_scheduler = True,
scheduler_final_lr = 1e-5,
step_size_lr = 45,
)

In [28]:
model = biolord.Biolord(
    adata=adata,
    n_latent=256,
    model_name="sciplex3",
    module_params=module_params,
    train_classifiers=False,
    split_key="split",
)

[rank: 0] Seed set to 0


In [29]:
model.train(
    max_epochs=200,
    batch_size=512,
    plan_kwargs=trainer_params,
    early_stopping=True,
    early_stopping_patience=20,
    check_val_every_n_epoch=10,
    num_workers=10,
    enable_checkpointing=False
)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_biol ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_biol ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/

Epoch 2/200:   0%|          | 1/200 [00:51<2:48:30, 50.81s/it, v_num=1, val_generative_mean_accuracy=0.102, val_generative_var_accuracy=-60.8, val_biolord_metric=-30.4, val_LOSS_KEYS.RECONSTRUCTION=326, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=232]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, an

Epoch 3/200:   1%|          | 2/200 [01:30<2:25:50, 44.19s/it, v_num=1, val_generative_mean_accuracy=0.336, val_generative_var_accuracy=-10.7, val_biolord_metric=-5.17, val_LOSS_KEYS.RECONSTRUCTION=290, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=211, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=212, unknown_attribute_penalty_loss=1.03e+5]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, an

...

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, an

Epoch 37/200:  18%|█▊        | 36/200 [24:00<1:49:14, 39.97s/it, v_num=1, val_generative_mean_accuracy=0.975, val_generative_var_accuracy=0.951, val_biolord_metric=0.963, val_LOSS_KEYS.RECONSTRUCTION=208, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=2.7, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=155, unknown_attribute_penalty_loss=1.02e+5]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, an

Epoch 38/200:  18%|█▊        | 37/200 [24:40<1:48:36, 39.98s/it, v_num=1, val_generative_mean_accuracy=0.979, val_generative_var_accuracy=0.955, val_biolord_metric=0.967, val_LOSS_KEYS.RECONSTRUCTION=207, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=2.28, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=155, unknown_attribute_penalty_loss=1.02e+5]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, an

In [31]:
def bool2idx(x):
    """
    Returns the indices of the True-valued entries in a boolean array `x`
    """
    return np.where(x)[0]

def repeat_n(x, n):
    """
    Returns an n-times repeated version of the Tensor x,
    repetition dimension is axis 0
    """
    # copy tensor to device BEFORE replicating it n times
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return x.to(device).view(1, -1).repeat(n, 1)


In [37]:
idx_test_control = np.where(
    (adata.obs["split"] == "test") & (adata.obs["control"] == 1)
)[0]

adata_test_control = adata[idx_test_control].copy()

idx_ood = np.where((adata.obs["split"] == "ood"))[0]

adata_ood = adata[idx_ood].copy()
dataset_ood = model.get_dataset(adata_ood)

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        
INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


In [38]:
dataset_ood = model.get_dataset(adata_ood)

In [43]:
import torch
from tqdm import tqdm

In [45]:
dataset = dataset_ood

In [109]:
pert_categories_index = pd.Index(adata_ood.obs["condition"].values, dtype="category")
allowed_cell_lines = []
n_obs = 500

cl_dict = {
    torch.Tensor([0.]): "A549",
    torch.Tensor([1.]): "K562",
    torch.Tensor([2.]): "MCF7",
}

layer = "X" if "X" in dataset else "layers"
cell_lines = ["A549", "K562", "MCF7"]

predictions_dict = {}
for cell_drug_dose_comb in tqdm(
    np.unique(pert_categories_index.values)):

    cell_type_idx = np.unique(dataset["cell_type"][idx_all])
    assert len(cell_type_idx) == 1
    dataset_control = model.get_dataset(adata_test_control[adata_test_control.obs["_scvi_cell_type"]==cell_type_idx[0]])
    
    bool_category = pert_categories_index.get_loc(cell_drug_dose_comb)
    idx_all = bool2idx(bool_category)
    idx = idx_all[0]  
    dataset_comb = {}

    
    dataset_comb[layer] = dataset_control[layer].to(model.device)
    dataset_comb["ind_x"] = dataset_control["ind_x"].to(model.device)
    for key in dataset_control:
        if key not in [layer, "ind_x"]:
            dataset_comb[key] = repeat_n(dataset[key][idx, :], n_obs)

    stop = False
    for tensor, cl in cl_dict.items():
        if (tensor == dataset["cell_type"][idx]).all():
            if cl not in cell_lines:
                stop = True
    if stop:
        continue
        
    pred_mean, pred_std = model.module.get_expression(dataset_comb)
    samples = torch.normal(pred_mean, pred_std)

    predictions_dict[cell_drug_dose_comb] = samples
    break



  0%|          | 0/40 [00:00<?, ?it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


  0%|          | 0/40 [00:05<?, ?it/s]


In [ ]:
# either convert dict to adata layers, or just save it as a dictionary. the latter is fine, too.

In [110]:
predictions_dict

{'A549_Alvespimycin_(17-DMAG)_HCl_10.0': tensor([[ 2.5184e-01,  5.2117e-01,  1.3084e-01,  ..., -1.9506e-05,
           1.9830e-04, -5.7395e-04],
         [ 2.7082e-01,  7.6124e-01,  4.0564e-01,  ..., -1.7216e-05,
           2.0655e-04, -7.6847e-04],
         [ 1.7310e-01,  2.2424e-01,  3.2515e-01,  ..., -1.8057e-05,
           2.0517e-04, -1.0013e-03],
         ...,
         [ 1.5208e-01, -1.0546e-01, -5.5587e-02,  ..., -1.6713e-05,
           2.0690e-04, -9.8138e-04],
         [-2.1371e-01,  2.3126e-01,  6.0327e-02,  ..., -1.7787e-05,
           2.0536e-04, -1.3881e-03],
         [ 1.8769e-01,  3.1853e-02,  3.0973e-01,  ..., -1.7503e-05,
           2.0499e-04, -1.1228e-03]], device='cuda:0')}